In [48]:
import seaborn as sb
from scipy.integrate import quad
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import matplotlib as mpl
import scipy.stats as st
from scipy.stats import multivariate_normal as mvn

#setting default parameters for plots
sb.set()
sb.set_style("dark") #removing gridlines and darkening background to make more presentation friendly 
sb.set(rc = {'figure.figsize':(18,11)})
mpl.rcParams['figure.dpi'] = 300

In [68]:
def vasicek(a,PD,N):
    Z=np.random.normal() 
    Y_i=np.random.normal(size=(1,N)) 
    X_i=np.sqrt(a)*Z+np.sqrt(1-a)*Y_i 
    Threshold=st.norm.ppf(PD) 
    Defaults = np.where(X_i<Threshold,1,0) 
    Loss = Defaults 
    return np.sum(Loss)

def MC(a,PD,N): #monte carlo function
    return np.array([vasicek(a,PD,N) for i in range(10000)])

def loss_density(x): 
    first= np.sqrt((1-a)/a)
    exp=(-1/(2*a))*(np.sqrt(1-a)*st.norm.ppf(x)-st.norm.ppf(PD))**2+0.5*(st.norm.ppf(x)**2) # exponent term of expontential
    return first*np.exp(exp)


def loss_density_integrand(x): #calculating first moment
    first= np.sqrt((1-a)/a)
    exp=(-1/(2*a))*(np.sqrt(1-a)*st.norm.ppf(x)-st.norm.ppf(PD))**2+0.5*(st.norm.ppf(x)**2) # exponent term of expontential
    return x*first*np.exp(exp)

def loss_density_variance_integrand(x): #calculating second moment
    first= np.sqrt((1-a)/a)
    exp=(-1/(2*a))*(np.sqrt(1-a)*st.norm.ppf(x)-st.norm.ppf(PD))**2+0.5*(st.norm.ppf(x)**2) # exponent term of expontential
    return (x**2)*first*np.exp(exp)

"""Computing Confidence intervals for estimates to perform repeated valuations to estimate the confidence intervals for our simulation estimates
we run the Monte Carlo simulation several times and compute the expected loss etc for each simulation we then store these values in a list. 
This process is repeated for each portfolio size
"""
def repeated_val(a,PD,N):
    List=[]
    for i in range(50): 
        temp = MC(a,PD,N) #this increases computational time exponentially
        Mean=np.mean(temp)
        Std=np.std(temp)
        VaR=sorted(temp)[9990]
        ES=np.mean(temp[temp>=VaR])
        EC=VaR-Mean
        List.append((N,Mean,Std,VaR,ES,EC))
    return List



In [50]:
"""Generates dataframe with Losses for portfolio of various sizes"""
N=np.array([1000,10000,50000,100000])
Loss=pd.DataFrame()
PD=0.05
a=0.1

In [ ]:
"""Generates plots of smaller portfolio sizes to show how quickly the model converges to its asymptotic form"""
fig,ax=plt.subplots(2,2)
sb.histplot(MC(a,PD,50),discrete=True, kde=True,ax=ax[0][0], color='magenta').set(title='50 Borrowers')
sb.histplot(MC(a,PD,60),discrete=True, kde=True,ax=ax[0][1],color='orange').set(title='60 Borrowers')
sb.histplot(MC(a,PD,80),discrete=True, kde=True,ax=ax[1][0],color='blue').set(title='80 Borrowers')
sb.histplot(MC(a,PD,120),discrete=True, kde=True,ax=ax[1][1],color='green').set(title='120 Borrowers');

In [51]:
"""I've split the dataframe call from this block because these take some time to run"""
Loss['1,000 Borrowers']=MC(0.1,0.05,N[0])
Loss['10,000 Borrowers']=MC(0.1,0.05,N[1])
Loss['50,000 Borrowers']=MC(0.1,0.05,N[2])
Loss['100,000 Borrowers']=MC(0.1,0.05,N[3])

In [ ]:
(
sb.histplot(Loss['1,000 Borrowers'],discrete=False,kde=True, color='blue',bins=100)
    .set(title='Loss Distribution for 1,000')
)
plt.xlabel('Loss')
plt.ylabel('');

In [ ]:
sb.histplot(Loss['10,000 Borrowers'],discrete=False,kde=True, color='magenta').set(title='Loss Distribution for 10,000');
plt.xlabel('Loss')
plt.ylabel('');

In [ ]:
(
sb.histplot(Loss['50,000 Borrowers'],discrete=False,kde=True, color='red',facecolor='black')
.set(title='Loss Distribution for 50,000')
);
plt.xlabel('Loss')
plt.ylabel('');

In [ ]:
sb.histplot(Loss['100,000 Borrowers'],discrete=False,kde=True, color='green').set(title='Loss Distribution for 100,000');
plt.xlabel('Loss')
plt.ylabel('');

In [ ]:
"""Comparative historgrams with losses represented as a fraction of borrowers"""
(
sb.histplot(Loss/N, multiple='dodge',bins=25,kde=True,color='palette')
.set(title='Loss Distribution')
)
#sb.kdeplot('Loss/N')
plt.xlabel('Loss')
plt.ylabel('');

In [ ]:
"""This shows two ways of calculating the standard deviation or variance of this distribution. 
The first is used because it is several orders of magnitudes faster (86 microseconds vs 36 milliseconds"""
mean = np.array([0,0]) #setting mean to 0
covariance = np.array([[1, 0.1],[0.1, 1]]) #setting variance to 1 with correlation 0.1
dist = mvn(mean=mean, cov=covariance) #setting parameter of distribution
np.sqrt(dist.cdf(np.array([Threshold,Threshold]))-0.05**2) #calculating variance of loss distribution

"""This is an alternative method of calculating the variance using moments"""
np.sqrt(quad(loss_density_variance_integrand,0,1)[0]-0.05**2)

In [52]:
"""Computing statistics for each portfolio size. Here, I compare the estimated portfolio qualities
to the asymptotitic qualities of Vasicek Model. Refer to the paper for derivations of formulas"""
Estimates=pd.DataFrame(index=['1,000 Borrowers','10,000 Borrowers','50,000 Borrowers','100,000 Borrowers'])
Estimates['Est. Exp. Loss']=Loss.mean()/np.sqrt(1-0.1)
Estimates['Act. Exp. Loss']=N*0.05 #actual expected loss via closed form formula

Estimates['Est. Standard Deviation']=Loss.std()
Estimates['Act. Standard Deviation']=np.sqrt(dist.cdf(np.array([Threshold,Threshold]))-0.05**2)*N


Estimates['Est. 99.9% VaR']=(
    
Loss.sort_values(by=['1,000 Borrowers']).iloc[9990]['1,000 Borrowers'],
Loss.sort_values(by=['10,000 Borrowers']).iloc[9990]['10,000 Borrowers'],
Loss.sort_values(by=['50,000 Borrowers']).iloc[9990]['50,000 Borrowers'],
Loss.sort_values(by=['100,000 Borrowers']).iloc[9990]['100,000 Borrowers'],
    
)
Estimates['Act. 99.9% VaR']=(st.norm.cdf((st.norm.ppf(0.05)+np.sqrt(0.1)*st.norm.ppf(0.999))/np.sqrt(1-0.1)))*N 

Estimates['Est. Economic Capital']=-Estimates['Est. Exp. Loss']+Estimates['Est. 99.9% VaR']
Estimates['Act. Economic Capital']=-Estimates['Act. Exp. Loss']+Estimates['Act. 99.9% VaR']

In [53]:
"""Estimating Expected Shortfall by integration"""

#VaR normalized by dividing by portfolio size
VaR_Norm=st.norm.cdf((st.norm.ppf(PD)+np.sqrt(a)*st.norm.ppf(0.999))/np.sqrt(1-a)) 

ES=quad(loss_density_integrand,0.2408,1)[0]/0.001 #integrating loss denisty function to calculate ES
Estimates['Act. Expected Shortfall']=ES*N

VaR_1K=st.norm.cdf((st.norm.ppf(0.05)+np.sqrt(0.1)*st.norm.ppf(0.999))/np.sqrt(1-0.1))*1000
VaR_10K=st.norm.cdf((st.norm.ppf(0.05)+np.sqrt(0.1)*st.norm.ppf(0.999))/np.sqrt(1-0.1))*10000
VaR_50K=st.norm.cdf((st.norm.ppf(0.05)+np.sqrt(0.1)*st.norm.ppf(0.999))/np.sqrt(1-0.1))*50000
VaR_100K=st.norm.cdf((st.norm.ppf(0.05)+np.sqrt(0.1)*st.norm.ppf(0.999))/np.sqrt(1-0.1))*100000

Estimates['Est. Expected Shortfall']=[
    Loss.query("`1,000 Borrowers` > @VaR_1K")['1,000 Borrowers'].mean(),
    Loss.query("`10,000 Borrowers` > @VaR_10K")['10,000 Borrowers'].mean(),
    Loss.query("`50,000 Borrowers` > @VaR_50K")['50,000 Borrowers'].mean(),
    Loss.query("`100,000 Borrowers` > @VaR_100K")['100,000 Borrowers'].mean()
    
]
Estimates.round(4)

,Est. Exp. Loss,Act. Exp. Loss,Est. Standard Deviation,Act. Standard Deviation,Est. 99.9% VaR,Act. 99.9% VaR,Est. Economic Capital,Act. Economic Capital,Act. Expected Shortfall,Est. Expected Shortfall
"1,000 Borrowers",52.5395,50.0,35.3337,34.8251,235,240.7941,182.4605,190.7941,271.1156,276.1250
"10,000 Borrowers",528.6602,500.0,347.9989,348.2512,2455,2407.9407,1926.3398,1907.9407,2711.1562,2912.2727
"50,000 Borrowers",2601.2274,2500.0,1724.0124,1741.2561,11849,12039.7037,9247.7726,9539.7037,13555.7809,14640.3333
"100,000 Borrowers",5308.2932,5000.0,3495.4547,3482.5122,24413,24079.4075,19104.7068,19079.4075,27111.5619,27917.5000


In [ ]:
OneK=repeated_val(a,PD,1000)

In [ ]:
TenK=repeated_val(a,PD,10000)

In [ ]:
FiftyK=repeated_val(a,PD,50000)

In [ ]:
HunK=repeated_val(a,PD,100000)

In [ ]:
"""Collecting results from the repeated valuations and grouping them by portfolio size"""
data=OneK+TenK+FiftyK+HunK
RepVal=pd.DataFrame(data,columns=['Borrowers','Mean','Standard Deviation', 'VaR', 'ES','EC']).set_index('Borrowers')
# CI.set_index('Borrowers')
B_1k=RepVal.groupby('Borrowers').get_group(1000)
B_10k=RepVal.groupby('Borrowers').get_group(10000)
B_50k=RepVal.groupby('Borrowers').get_group(50000)
B_100k=RepVal.groupby('Borrowers').get_group(100000)

In [ ]:
"""Calculating Confidence intervals for each statistic and each size portfolio"""
CI=pd.DataFrame(index=['1,000','10,000','50,000','100,000'])

CI['Expected Loss']=(
    st.norm.interval(0.95,loc=B_1k['Mean'].mean(), scale=B_1k['Mean'].std()),
    st.norm.interval(0.95,loc=B_10k['Mean'].mean(), scale=B_10k['Mean'].std()),
    st.norm.interval(0.95,loc=B_50k['Mean'].mean(), scale=B_50k['Mean'].std()),
    st.norm.interval(0.95,loc=B_100k['Mean'].mean(), scale=B_100k['Mean'].std())
)

CI['Standard Deviation']=(
    st.norm.interval(0.95,loc=B_1k['Standard Deviation'].mean(), scale=B_1k['Standard Deviation'].std()),
    st.norm.interval(0.95,loc=B_10k['Standard Deviation'].mean(), scale=B_10k['Standard Deviation'].std()),
    st.norm.interval(0.95,loc=B_50k['Standard Deviation'].mean(), scale=B_50k['Standard Deviation'].std()),
    st.norm.interval(0.95,loc=B_100k['Standard Deviation'].mean(), scale=B_100k['Standard Deviation'].std())
)

CI['99.9% VaR']=(
    st.norm.interval(0.95,loc=B_1k['VaR'].mean(), scale=B_1k['VaR'].std()),
    st.norm.interval(0.95,loc=B_10k['VaR'].mean(), scale=B_10k['VaR'].std()),
    st.norm.interval(0.95,loc=B_50k['VaR'].mean(), scale=B_50k['VaR'].std()),
    st.norm.interval(0.95,loc=B_100k['VaR'].mean(), scale=B_100k['VaR'].std())
)

CI['Economic Capital']=(
    st.norm.interval(0.95,loc=B_1k['EC'].mean(), scale=B_1k['EC'].std()),
    st.norm.interval(0.95,loc=B_10k['EC'].mean(), scale=B_10k['EC'].std()),
    st.norm.interval(0.95,loc=B_50k['EC'].mean(), scale=B_50k['EC'].std()),
    st.norm.interval(0.95,loc=B_100k['EC'].mean(), scale=B_100k['EC'].std())
    )

CI['Expected Shortfall']=(
    st.norm.interval(0.95,loc=B_1k['ES'].mean(), scale=B_1k['ES'].std()),
    st.norm.interval(0.95,loc=B_10k['ES'].mean(), scale=B_10k['ES'].std()),
    st.norm.interval(0.95,loc=B_50k['ES'].mean(), scale=B_50k['ES'].std()),
    st.norm.interval(0.95,loc=B_100k['ES'].mean(), scale=B_100k['ES'].std())
)

#rounding each column since they contain tuples (immutable lists) and a tradition pandas round function won't work
CI['Expected Loss'] = CI['Expected Loss'].apply(lambda x: np.round(x,4))
CI['Standard Deviation'] = CI['Standard Deviation'].apply(lambda x: np.round(x,4))
CI['99.9% VaR'] = CI['99.9% VaR'].apply(lambda x: np.round(x,4))
CI['Expected Shortfall'] = CI['Expected Shortfall'].apply(lambda x: np.round(x,4))
CI['Economic Capital'] = CI['Economic Capital'].apply(lambda x: np.round(x,4))

In [54]:
%timeit Loss.query("`1,000 Borrowers` > @VaR_1K")['1,000 Borrowers'].mean()

2.04 ms ± 94.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [57]:
test = np.array(Loss['1,000 Borrowers'])

In [59]:
%timeit np.mean(test[test>=40])

84.7 µs ± 757 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [60]:
Loss['1,000 Borrowers']>=40

0       False
1       False
2       False
3       False
4        True
        ...  
9995    False
9996    False
9997    False
9998     True
9999     True
Name: 1,000 Borrowers, Length: 10000, dtype: bool

In [71]:
%timeit -n1 repeated_val(a,PD,10)

KeyboardInterrupt: 

In [66]:
len(MC(a,PD,100))

10000